# 유사한 단어 찾기 게임

1. 사전학습된 모델 또는 적절한 데이터 셋을 찾는다. 
2. 워드 임베딩 모델을 학습 시킨다. 
3. 단어 유사도가 0.8 이상인 A,B를 랜덤 추출한다. 
4. A,B가 대응되는 C를 추출한다.
5. D를 입력받는다. 

=> <br>
A:B = C:D 관계에 대응하는 
D를 찾는 게임 



<출력 예시>

관계 [수긍 : 추락 = 대사관 : ?] <br>
모델이 예측한 가장 적합한 단어 : 잠입 <br>
당신의 답변과 모델 예측의 유사도 : 0.34 <br>
아쉽네요. 더 생각해보세요

_________________

### 데이터 set

url : https://www.aihub.or.kr/aihubdata/data/view.do?pageIndex=5&currMenu=115&topMenu=100&srchOptnCnd=OPTNCND001&searchKeyword=&srchDetailCnd=DETAILCND001&srchOrder=ORDER001&srchPagePer=20&srchDataRealmCode=REALM002&aihubDataSe=data&dataSetSn=71304

# 1. 데이터 준비

In [1]:
# 데이터 불러오기 


import os
import glob
import pandas as pd

base_dir = "data"  # 최상위 폴더

# 하위 폴더까지 .tsv 전부 검색 (대소문자 대응)
paths = glob.glob(os.path.join(base_dir, "**", "*.tsv"), recursive=True)
paths += glob.glob(os.path.join(base_dir, "**", "*.TSV"), recursive=True)

print(f"찾은 TSV 파일 수: {len(paths)}")
for p in paths[:10]:
    print(" -", p)
if not paths:
    raise FileNotFoundError("data 폴더(및 하위폴더)에 .tsv 파일을 못 찾았습니다. 경로/확장자 확인하세요.")

dfs = []
for p in paths:
    try:
        df = pd.read_csv(p, sep="\t", encoding="utf-8")  # 필요시 encoding 변경
        # 출처 파일 표시(선택)
        df["__source_file__"] = os.path.relpath(p, base_dir)
        dfs.append(df)
    except Exception as e:
        print(f"[스킵] {p} -> {e}")

if not dfs:
    raise ValueError("읽을 수 있는 TSV가 없습니다. 파일 인코딩/구분자/내용을 확인하세요.")

final_df = pd.concat(dfs, ignore_index=True, sort=False)

찾은 TSV 파일 수: 14622
 - data/TS_배움과 학문/Knowledge_배움과 학문_885.tsv
 - data/TS_배움과 학문/Knowledge_배움과 학문_891.tsv
 - data/TS_배움과 학문/Knowledge_배움과 학문_649.tsv
 - data/TS_배움과 학문/Knowledge_배움과 학문_661.tsv
 - data/TS_배움과 학문/Knowledge_배움과 학문_107.tsv
 - data/TS_배움과 학문/Knowledge_배움과 학문_113.tsv
 - data/TS_배움과 학문/Knowledge_배움과 학문_675.tsv
 - data/TS_배움과 학문/Knowledge_배움과 학문_1322.tsv
 - data/TS_배움과 학문/Knowledge_배움과 학문_846.tsv
 - data/TS_배움과 학문/Knowledge_배움과 학문_852.tsv


In [2]:
final_df = final_df[['utterance_id', 'text']]

# 2. 데이터 전처리 

In [3]:
import re
import pandas as pd
import re
from konlpy.tag import Okt
okt = Okt()
from tqdm import tqdm
from collections import Counter

In [4]:
# 불용어 파일
ko_stopwords = pd.read_csv("ko_stopwords.txt", header=None, names=["word"])
ko_stopwords = ko_stopwords["word"].tolist()

In [5]:
# 추가 불용어 리스트
extra_noise_nouns = {

    "리다","이다","하다","인지","이란","이나","가세",
    "정말","역시","현재","지난","이후","가지","기간","정도",

}

okt = Okt()
def clean_text_to_nouns(text: str):
    # (a) NaN → 빈 리스트
    if pd.isna(text):
        return []

    # 문자열 강제 변환
    text = str(text)

    # (b) 간단 정제
    #  - 한글/영문/숫자/공백/기본 문장부호만 남기기
    text = re.sub(r"[^0-9a-zA-Z가-힣\s\.!?]", " ", text)
    #  - 연속 공백 하나로 축소
    text = re.sub(r"\s+", " ", text).strip()

    # (c) 명사만 추출
    nouns = okt.nouns(text) 

    # (d) 후처리 필터링
    out = []
    for t in nouns:
        t = t.strip()
        # 길이 1 이하 제거 (예: "간", "인" 등)
        if len(t) <= 1:
            continue
        # 불용어/추가 잡음 제거
        if t in ko_stopwords or t in extra_noise_nouns:
            continue
        out.append(t)

    return out

final_df["clean_nouns"] = [clean_text_to_nouns(t) for t in final_df["text"]]


In [6]:
# 추가 불용어 집합
extra_noise_nouns = {
    "리다","이다","하다","인지","이란","이나","가세",
    "정말","역시","현재","지난","이후","가지","기간","정도",'이전','원래','사실','가장','건가','반면','대신','멍하니','누가','누가','대해','관련','는걸','서서','여러','통해'
}

def remove_extra_noise(nouns_list):

    return [t for t in nouns_list if t not in extra_noise_nouns]


final_df["clean_nouns"] = [remove_extra_noise(n) for n in final_df["clean_nouns"]]

# 3. 데이터셋 학습

In [7]:
from gensim.models import Word2Vec

sentences = final_df["clean_nouns"].tolist()
model = Word2Vec(
    sentences=sentences,
    vector_size=100,
    window=5,
    sg=0,
)

# 4. 게임 함수 

In [8]:
import random

def make_A_B(sim = 0.8, max_trials =100):
    for _ in range(max_trials):
        A = random.choice(list(model.wv.key_to_index.keys()))
        try: 
            sims_word = model.wv.most_similar(A, topn=200)
            sims_list = [w for (w, s) in sims_word if s >= sim and w != A and w in model.wv.key_to_index]
            #if not sims_list:       
            #    continue
            B = random.choice(sims_list)
            return A, B
        except KeyError: 
            continue
    return None, None



In [9]:

def get_analogy( A, B, C, topn=1):
    result = model.wv.most_similar(positive=[B, C], negative=[A], topn=topn)
    return result[0]




In [10]:


def play_game():
    global cont
    print("게임 시작!\n")

    while True:  # 전체 게임 루프
        A, B = make_A_B()
        #words = list(model.wv.key_to_index.keys())
        sims = model.wv.most_similar(A, topn=200)
        cand_words = [w for (w, s) in sims if w not in {A, B}]  # 단어만 추출

        C = None
        predicted_D = None
        score = None
            
        for w in cand_words[:5]:
            pred, sc = get_analogy(A, B, w)
            if pred in model.wv.key_to_index: 
                C, predicted_D, score = w, pred, sc
                break

            
            predicted_D, score = get_analogy(A, B, C)
            # 예측 단어도 vocab에 확실히 있는지 점검
            if predicted_D in model.wv.key_to_index:
                break

        print(f'관계 [ {A} : {B} = {C} : ? ]')
        while True: 
            answer_input = input("정답은? (정답확인 : 포기) : ").strip()

            # 정답 공개
            if answer_input == '포기':
                print(f"정답: {predicted_D} (score={score:.4f})")
                break

            # vocab
            if answer_input not in model.wv.key_to_index:
                print("입력한 단어는 어휘에 없습니다. !!!!!!!retry!!!!!!!!")
                continue

            # 유사도
            similarity = model.wv.similarity(answer_input, predicted_D)
            print(f"입력값과 정답의 유사도: {similarity:.4f}")

            if answer_input == predicted_D:
                print("정답입니다!")
                break
            else:
                print("땡!!!!!!!!!!!!!!! (정답 : 포기)")


        cont = input("\n다음 문제 GO!!!!!!!!!!!!! (y/엔터=계속, n=종료): ").strip().lower()
        if cont in {"n", "no"}:
            print("게임 종료")
            break
        #print()

In [11]:
while True:
    play_game()
    if 'cont' in globals() and cont in {'n', 'no'}:
        print('====! 빠이염 !====')
        break

게임 시작!

관계 [ 신진대사 : 전해질 = 촉진 : ? ]
입력한 단어는 어휘에 없습니다. !!!!!!!retry!!!!!!!!
입력한 단어는 어휘에 없습니다. !!!!!!!retry!!!!!!!!
입력한 단어는 어휘에 없습니다. !!!!!!!retry!!!!!!!!
입력한 단어는 어휘에 없습니다. !!!!!!!retry!!!!!!!!
입력한 단어는 어휘에 없습니다. !!!!!!!retry!!!!!!!!
입력값과 정답의 유사도: 0.7631
땡!!!!!!!!!!!!!!! (정답 : 포기)
입력값과 정답의 유사도: 0.2579
땡!!!!!!!!!!!!!!! (정답 : 포기)
입력한 단어는 어휘에 없습니다. !!!!!!!retry!!!!!!!!
입력한 단어는 어휘에 없습니다. !!!!!!!retry!!!!!!!!
입력값과 정답의 유사도: 0.7631
땡!!!!!!!!!!!!!!! (정답 : 포기)
입력값과 정답의 유사도: 0.7589
땡!!!!!!!!!!!!!!! (정답 : 포기)
입력값과 정답의 유사도: 0.2408
땡!!!!!!!!!!!!!!! (정답 : 포기)
정답: 위산 (score=0.8200)
게임 종료
====! 빠이염 !====
